In [58]:
import pandas as pd
from bs4 import BeautifulSoup
import requests

df_player_stats = pd.DataFrame()
df_match_stats = pd.DataFrame()


In [65]:
for i in range(539,3177):
    url = 'https://siege.gg/matches/'+str(i)
    html_content = requests.get(url).text
    soup = BeautifulSoup(html_content, 'lxml')
    print(url)
    
    index = url.split('https://siege.gg/matches/')[1].split('-')[0]
    
    if soup.time == None:
        continue
    time_str = soup.time.text
    date = time_str.split(', ')[1]

    classes = ['sp__player','sp__rating','sp__kills','sp__ok','sp__kost','sp__kpr','sp__srv','sp__1vx','sp__plant','sp__hs','sp__atk','sp__def']

    stats_list = {}

    for section in classes:
        stats_list[section[4:]] = []
        for content in soup.find_all('td',section):
            stats_list[section[4:]].append(content.text.replace('\n',''))
    if len(stats_list['player']) != 0:
        df_player_stats_temp = pd.DataFrame(stats_list)
        df_player_stats = pd.concat([df_player_stats,df_player_stats_temp], sort=False)
    stats_list['date'] = []
    for i in range(len(stats_list['player'])):
        stats_list['date'].append(date)
        
    team_stats = {}
    team_stats['Date'] = date

    title = soup.find('title').text
    team1 = title.split(' vs')[0]
    team2 = title.split('vs ')[1].split(',')[0]

    team_stats['Team 1'] = team1
    team_stats['Team 2'] = team2


    map_bans = soup.find_all('tr','ban__map__row--ban')
    for i in range(len(map_bans)):
        map_bans[i] = map_bans[i].contents
        for j in range(len(map_bans[i])):
            if j >= len(map_bans[i]):
                break
            if map_bans[i][j] == '\n':
                map_bans[i].pop(j)
            else:
                map_bans[i][j] = map_bans[i][j].text

    for i in range(len(map_bans)):
        for j in range(len(map_bans[i])):
            map_bans[i][j] = map_bans[i][j].text

    ban_dict = {}
    t1_ban_count = 0 #Count of number of bans of Team 1
    t2_ban_count = 0 #Count of number of bans of Team 2
    t1_pick_count = 0 #Count of number of picks of Team 1
    t2_pick_count = 0 ##Count of number of bans of Team 2

    for map_ban_object in map_bans:
        if map_ban_object[0] == 'Ban':
            t1_ban_count += 1
            ban_dict["Team 1 Ban "+str(t1_ban_count)] = map_ban_object[1]
        if map_ban_object[0] == 'Pick':
            t1_pick_count += 1
            ban_dict["Team 1 Pick "+str(t1_pick_count)] = map_ban_object[1]
        if map_ban_object[0] == '':
            if map_ban_object[2] == 'Ban':
                t2_ban_count += 1
                ban_dict["Team 2 Ban "+str(t2_ban_count)] = map_ban_object[1]
            if map_ban_object[2] == 'Pick':
                t2_pick_count += 1
                ban_dict["Team 2 Pick "+str(t2_pick_count)] = map_ban_object[1]
        if map_ban_object[0] == 'Decider':
            ban_dict["decider"] = map_ban_object[1]

    dict1 = dict(sorted(ban_dict.items()))

    for keys in dict1:
        team_stats[keys] = dict1[keys]

    round_logs = soup.find_all("li","log__line")
    formatted_round_logs = []
    for i in range(len(round_logs)):
        round_object = []
        round_logs[i] = round_logs[i].text.replace('\n','')
        round_logs[i] = round_logs[i].split(' wins ')
        round_object.append(round_logs[i][0])
        if len(round_logs[i][0].split('tie')) == 2:
               continue
        round_logs[i][1] = round_logs[i][1].split(' attack by ')
        round_object.append(round_logs[i][1][0])
        if len(round_logs[i][1]) == 2:
            round_object.append(round_logs[i][1][1])
        if len(round_object) == 2 and len(round_object[1]) > 15:
            temp_str = round_object[1]
            temp_str = temp_str.split(' defense by ')
            round_object[1] = temp_str[0]
            if len(temp_str) == 1:
                continue
            round_object.append(temp_str[1])
        formatted_round_logs.append(round_object)

    map_count = 1
    round_count = 1
    for i in range(len(formatted_round_logs)):
        if len(formatted_round_logs[i]) == 3:
            team_stats["Map "+str(map_count)+" Round "+str(round_count)+" Winner"] = formatted_round_logs[i][0]
            team_stats["Map "+str(map_count)+" Round "+str(round_count)+" Bombsite"] = formatted_round_logs[i][1]
            team_stats["Map "+str(map_count)+" Round "+str(round_count)+" Win Type"] = formatted_round_logs[i][2]
            round_count+=1
        else:
            map_count += 1
            round_count = 0
    
    df_match_stats_temp = pd.DataFrame(team_stats,index=[index])
    df_match_stats = pd.concat([df_match_stats,df_match_stats_temp], sort=False)

https://siege.gg/matches/539
https://siege.gg/matches/540
https://siege.gg/matches/541
https://siege.gg/matches/542
https://siege.gg/matches/543
https://siege.gg/matches/544
https://siege.gg/matches/545
https://siege.gg/matches/546
https://siege.gg/matches/547
https://siege.gg/matches/548
https://siege.gg/matches/549
https://siege.gg/matches/550
https://siege.gg/matches/551
https://siege.gg/matches/552
https://siege.gg/matches/553
https://siege.gg/matches/554
https://siege.gg/matches/555
https://siege.gg/matches/556
https://siege.gg/matches/557
https://siege.gg/matches/558
https://siege.gg/matches/559
https://siege.gg/matches/560
https://siege.gg/matches/561
https://siege.gg/matches/562
https://siege.gg/matches/563
https://siege.gg/matches/564
https://siege.gg/matches/565
https://siege.gg/matches/566
https://siege.gg/matches/567
https://siege.gg/matches/568
https://siege.gg/matches/569
https://siege.gg/matches/570
https://siege.gg/matches/571
https://siege.gg/matches/572
https://siege.

In [66]:
df_match_stats.shape

(2869, 262)

In [67]:
df_player_stats.shape

(25187, 12)

In [69]:
df_match_stats.to_csv('SiegeGG Match Info.csv')

In [70]:
df_player_stats.to_csv('SiegeGG Player Info.csv')